In [2]:
import cv2
from matplotlib import category
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
# 1. 加载发票分类标准
def load_classification_standard(file_path):
    """
    加载发票分类表格，并将其转化为字典或数据框以供后续分类使用。
    """
    classification_df = pd.read_excel(file_path)
    # 修改列名
    classification_df.columns = ["category_id", "category", "items"]
    # 根据实际表格结构选择加载方式
    return classification_df


classification_file_path = "./whu/classification_standard.xlsx"
classification_standard = load_classification_standard(
        classification_file_path)


In [4]:
classification_standard.head()

,category_id,category,items
0,3020101,商品和服务支出_办公费_办公用品,计算器、纸类、本册、便签、财会用品、画具画材、签字笔、钢笔、笔芯、开关、电料、抽纸、卷纸、手...
1,3020102,商品和服务支出_其他商品和服务支出_书报杂志,各类报刊杂志
2,3020190,商品和服务支出_办公费_其他,洗洁精、油污清洁剂、除湿干燥剂、洁厕剂、家电清洁用品、消毒液、地板清洁剂、一次性清洁用品、拖...
3,3021302,商品和服务支出_维修(护)费_设备维修费,CPU、显卡、内存、主板、散热器、光驱、机箱、显示器、电源声卡、扩展卡、装机配件、SSD固态...
4,3021305,商品和服务支出_维修(护)费_网络运行与维护费,数据库、操作系统、防病毒、办公软件等


In [5]:
def build_classification_context(classification_df):
    """
    构建分类的上下文信息，将表格转换为大模型的输入上下文。
    """
    context = ""
    for index, row in classification_df.iterrows():
        # 假设表格中包含 "商品名称" 和 "分类" 列
        context += f"相关对应商品: {row['items']} -> 科目名称: {row['category']}\n"

    print(context)
    return context

# 构建分类上下文
classification_context = build_classification_context(classification_standard)



相关对应商品: 计算器、纸类、本册、便签、财会用品、画具画材、签字笔、钢笔、笔芯、开关、电料、抽纸、卷纸、手帕纸、湿巾、垃圾袋、垃圾桶、水壶、电话机、手电筒、电子钟表、电子灭蚊器、收音机、万用遥控器、对讲机、塑料杯、保温杯、杯具等 -> 科目名称: 商品和服务支出_办公费_办公用品
相关对应商品: 各类报刊杂志 -> 科目名称: 商品和服务支出_其他商品和服务支出_书报杂志
相关对应商品: 洗洁精、油污清洁剂、除湿干燥剂、洁厕剂、家电清洁用品、消毒液、地板清洁剂、一次性清洁用品、拖把、扫把、脸盆、水桶、抹布、百洁布、手套、鞋套、清洁刷具、其它清洁工具、清洁用具配件、饮用水等 -> 科目名称: 商品和服务支出_办公费_其他
相关对应商品: CPU、显卡、内存、主板、散热器、光驱、机箱、显示器、电源声卡、扩展卡、装机配件、SSD固态硬盘、移动硬盘、UPS电源、线缆、电脑工具、电脑清洁、插座、键盘、鼠标、摄像头、U盘、硬盘盒、手写板、鼠标垫、网络仪表仪器、投影配件、会议音频、硒鼓、墨粉、墨盒、墨水、色带、CD-R碟片、DVDR碟片、可擦写碟片、光盘包、光盘袋、光盘箱、光盘笔、光盘帖、迷你音响、LED屏配件、耳机、耳麦、麦克风、专业音频、苹果配件、无线音箱、存储卡、读卡器、相机包、电池、充电器、路由器、网络机柜、无线路由、无线网卡、网络存储、网络配件、防火墙等 -> 科目名称: 商品和服务支出_维修(护)费_设备维修费
相关对应商品: 数据库、操作系统、防病毒、办公软件等 -> 科目名称: 商品和服务支出_维修(护)费_网络运行与维护费
相关对应商品: 移液器、天平衡器、混合搅拌、清洗消毒、泵类仪器、干燥箱系列、培养箱系列、试验箱系列、振荡器、电炉系列、水浴水槽、分子生物学、萃取提纯、电泳成像、微生物学、粉碎合成、物理光学、光谱分析、色谱分析、PH计、酸度计、电导电阻、水质检测、多参数检测、环境监测、超纯水系统、净化安全、安全防护、急救清洗、石油仪器、食品检测、药物检测、煤炭纺织、特殊仪器、配件耗材、防噪音耳塞等 -> 科目名称: 商品和服务支出_专用材料费_实验室用品
相关对应商品: 羽毛球类、网球类、篮球类、乒乓类、足球类、桌球类、棒球类、铅球类、跳马、起跑器、跳高类、铁饼、标枪类、排球装备、其他体育用品 -> 科目名称: 商品和服务支出_专用材料费_消耗性体育用品
相

In [7]:
prompt = "根据发票上的商品名称，将其分类到相应的科目名称中。\n"
prompt += classification_context
print(prompt)

根据发票上的商品名称，将其分类到相应的科目名称中。
相关对应商品: 计算器、纸类、本册、便签、财会用品、画具画材、签字笔、钢笔、笔芯、开关、电料、抽纸、卷纸、手帕纸、湿巾、垃圾袋、垃圾桶、水壶、电话机、手电筒、电子钟表、电子灭蚊器、收音机、万用遥控器、对讲机、塑料杯、保温杯、杯具等 -> 科目名称: 商品和服务支出_办公费_办公用品
相关对应商品: 各类报刊杂志 -> 科目名称: 商品和服务支出_其他商品和服务支出_书报杂志
相关对应商品: 洗洁精、油污清洁剂、除湿干燥剂、洁厕剂、家电清洁用品、消毒液、地板清洁剂、一次性清洁用品、拖把、扫把、脸盆、水桶、抹布、百洁布、手套、鞋套、清洁刷具、其它清洁工具、清洁用具配件、饮用水等 -> 科目名称: 商品和服务支出_办公费_其他
相关对应商品: CPU、显卡、内存、主板、散热器、光驱、机箱、显示器、电源声卡、扩展卡、装机配件、SSD固态硬盘、移动硬盘、UPS电源、线缆、电脑工具、电脑清洁、插座、键盘、鼠标、摄像头、U盘、硬盘盒、手写板、鼠标垫、网络仪表仪器、投影配件、会议音频、硒鼓、墨粉、墨盒、墨水、色带、CD-R碟片、DVDR碟片、可擦写碟片、光盘包、光盘袋、光盘箱、光盘笔、光盘帖、迷你音响、LED屏配件、耳机、耳麦、麦克风、专业音频、苹果配件、无线音箱、存储卡、读卡器、相机包、电池、充电器、路由器、网络机柜、无线路由、无线网卡、网络存储、网络配件、防火墙等 -> 科目名称: 商品和服务支出_维修(护)费_设备维修费
相关对应商品: 数据库、操作系统、防病毒、办公软件等 -> 科目名称: 商品和服务支出_维修(护)费_网络运行与维护费
相关对应商品: 移液器、天平衡器、混合搅拌、清洗消毒、泵类仪器、干燥箱系列、培养箱系列、试验箱系列、振荡器、电炉系列、水浴水槽、分子生物学、萃取提纯、电泳成像、微生物学、粉碎合成、物理光学、光谱分析、色谱分析、PH计、酸度计、电导电阻、水质检测、多参数检测、环境监测、超纯水系统、净化安全、安全防护、急救清洗、石油仪器、食品检测、药物检测、煤炭纺织、特殊仪器、配件耗材、防噪音耳塞等 -> 科目名称: 商品和服务支出_专用材料费_实验室用品
相关对应商品: 羽毛球类、网球类、篮球类、乒乓类、足球类、桌球类、棒球类、铅球类、跳马、起跑器、跳高类、铁饼、标枪类、排球装备、其他体育用品 -> 科目名

In [ ]:
from zhipuai import ZhipuAI
api_key = ""  # 填写自己的APIKey
client = ZhipuAI(api_key=api_key) # 填写您自己的APIKey
response = client.chat.completions.create(
    model="glm-4-plus",  # 填写需要调用的模型编码
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": "商品名称: 电脑"},
    ],
)
print(response.choices[0].message)

CompletionMessage(content='根据您提供的分类信息，"电脑"这一商品名称可以归类到以下两个科目名称中，具体取决于电脑的用途和采购目的：\n\n1. **商品和服务支出_维修(护)费_设备维修费**：\n   - 如果电脑是作为办公设备的一部分，用于日常办公或特定项目的设备维护和升级，可以归类到这个科目。\n\n2. **商品和服务支出_其他商品和服务支出_小额设备（科研项目）** 或 **商品和服务支出_其他商品和服务支出_小额设备（非科研项目）**：\n   - 如果电脑是作为新的设备采购，用于科研项目，则归类到“小额设备（科研项目）”。\n   - 如果电脑是用于非科研项目的日常办公或其他用途，则归类到“小额设备（非科研项目）”。\n\n具体归类需要根据实际使用情况和采购目的来确定。', role='assistant', tool_calls=None)


In [1]:
import json
import re

def extract_json_from_response(response_text):
    """
    从大模型的回答中提取出 JSON 格式的数据。

    参数:
    response_text (str): 大模型的回答文本。

    返回:
    dict: 提取出的 JSON 数据，解析为 Python 字典。
    """
    # 使用正则表达式提取 JSON 部分
    json_pattern = re.compile(r'\{.*?\}', re.DOTALL)
    match = json_pattern.search(response_text)
    
    if match:
        json_str = match.group(0)
        try:
            # 解析 JSON 字符串为 Python 字典
            json_data = json.loads(json_str)
            return json_data
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
            return None
    else:
        print("未找到 JSON 格式的数据")
        return None

# 示例使用
response_text = """
根据您提供的商品详细信息“*电子元件*八通道ADC芯片AD9257S”，我们可以将其分类到以下科目名称中：

**科目名称: 商品和服务支出_维修(护)费_设备维修费**

理由：
- **电子元件**：这类商品通常用于电子设备的维修或升级。
- **八通道ADC芯片AD9257S**：这是一种特定的电子元件，用于数据采集和转换，常见于各种电子设备和系统中。

因此，将其归类到“设备维修费”科目是合理的，因为这类元件的采购通常与设备的维修、升级或维护相关。

如果该元件是用于特定的科研项目或设备的研发，也可以考虑归类到“商品和服务支出_其他商品和服务支出_小额设备（科研项目）”科目，但基于提供的信息，归类到“设备维修费”更为直接和符合常规分类逻辑。

```json
{
  "商品详细信息": "*电子元件*八通道ADC芯片AD9257S",
  "科目名称": "商品和服务支出_维修(护)费_设备维修费",
  "理由": [
    "电子元件：这类商品通常用于电子设备的维修或升级。",
    "八通道ADC芯片AD9257S：这是一种特定的电子元件，用于数据采集和转换，常见于各种电子设备和系统中。",
    "因此，将其归类到“设备维修费”科目是合理的，因为这类元件的采购通常与设备的维修、升级或维护相关。",
    "如果该元件是用于特定的科研项目或设备的研发，也可以考虑归类到“商品和服务支出_其他商品和服务支出_小额设备（科研项目）”科目，但基于提供的信息，归类到“设备维修费”更为直接和符合常规分类逻辑。"
  ]
}
"""

In [2]:
json_pattern = re.compile(r'\{.*?\}', re.DOTALL)
match = json_pattern.search(response_text)

In [9]:
json_str = match.group(0)
json_data = json.loads(json_str)

In [10]:
json_data

{'商品详细信息': '*电子元件*八通道ADC芯片AD9257S',
 '科目名称': '商品和服务支出_维修(护)费_设备维修费',
 '理由': ['电子元件：这类商品通常用于电子设备的维修或升级。',
  '八通道ADC芯片AD9257S：这是一种特定的电子元件，用于数据采集和转换，常见于各种电子设备和系统中。',
  '因此，将其归类到“设备维修费”科目是合理的，因为这类元件的采购通常与设备的维修、升级或维护相关。',
  '如果该元件是用于特定的科研项目或设备的研发，也可以考虑归类到“商品和服务支出_其他商品和服务支出_小额设备（科研项目）”科目，但基于提供的信息，归类到“设备维修费”更为直接和符合常规分类逻辑。']}

In [16]:
from fileinput import filename
import shutil
import os

def handle_upload(file):
    file_name = os.path.basename(file)
    print(file_name)
    file_path = f"./tmp/temp_{file_name}"
    file = open(file, "rb")
    with open(file_path, "wb") as buffer:
        shutil.copyfileobj(file, buffer)
    return file_path

handle_upload("./whu/test/3.jpg")

3.jpg


'./tmp/temp_3.jpg'

In [17]:
file = open("/private/var/folders/tw/5jgn7f8j74zgq_93zn_8f2n40000gn/T/gradio/15f1dfdcf4c9ae7fd8a84953c3c376784a0caf7c2570c0d87cde09430d40300c/ZW_00_2024_PZ_B1ACD36F91274FBA98B293A5475AF429_1BODY_00003.jpg", "rb")

In [20]:
file = "/private/var/folders/tw/5jgn7f8j74zgq_93zn_8f2n40000gn/T/gradio/15f1dfdcf4c9ae7fd8a84953c3c376784a0caf7c2570c0d87cde09430d40300c/3.jpg"
file_path = "/Users/aliancn/code/python/Financial-identification/tmp/3.jpg"
shutil.copy(file, file_path)

'/Users/aliancn/code/python/Financial-identification/tmp/3.jpg'

In [7]:
# # coding=gb2312

import pandas as pd
import re
import os


# Extract relevant columns for creating regex patterns
def load_classification_data():
    file_path = os.path.join(os.getcwd(), './whu/classification_standard.xlsx') 
    excel_data = pd.ExcelFile(file_path)
    sheet_1 = excel_data.parse('商品对照')
    return sheet_1[['科目名称', '相关对应商品（达不到资产验收条件的商品，且不满足设备附件验收条件）']]


categories = load_classification_data()


# Example text to match against the regex patterns
example_text = "维修费"


def regex_match(categories, example_text):
    # Generate regex patterns based on the '相关对应商品' field for each category
    regex_patterns = {}
    for index, row in categories.iterrows():
        category = row['科目名称']
        items = row['相关对应商品（达不到资产验收条件的商品，且不满足设备附件验收条件）']
        # Split items by common delimiters (Chinese comma, comma, and other separators)
        item_keywords = re.split(r'[，、,]', str(items))
        # Create a regex pattern by joining keywords with '|'
        pattern = '|'.join([re.escape(keyword.strip())
                           for keyword in item_keywords if keyword.strip()])
        regex_patterns[category] = pattern

    # Check which category regex matches the example text
    matching_categories = [category for category, pattern in regex_patterns.items(
    ) if re.search(pattern, example_text)]

    # If no category matched, return a default message
    return matching_categories if matching_categories else ["未找到匹配类别"]


# Display the matching categories
print(regex_match(categories, example_text))


['未找到匹配类别']
